## Part 2: Neural Networks


### Neurons
A neuron is just a container for weights, a bias, and an activation function. It takes a number of inputs, combines them, and produces a scalar output. The number of weights is determined by the number of inputs (features) to that neuron. The activation function allows for more complex mapping of inputs to the output (and so that layers of neurons don't reduce to a single linear combination). The code defining a neuron is here:

```julia
    mutable struct Neuron{T<:AbstractFloat,F<:Union{Function,Nothing}}
        w::Vector{Value{T}}
        b::Value{T}
        activation::F
    end
```
We can define a function to make variables of this type "callable": `n(X)`

```julia
    function (n::Neuron)(x)
        if length(x) != length(n.w)
            error("In calling n(x), expected $(length(n.w)) inputs to neuron, got $(length(x)):\n\tx = $(x)")
        end
        raw = sum(n.w .* x)+ n.b
        isnothing(n.activation) && return raw 
        return n.activation(raw)
    end
```
This just computes the inner product of X and the weights, adds b, and then runs the result through an activation function.

In [2]:
using Micrograd

## make a linear neuron with three inputs and call it
n = neuron(3,nothing)
x = [1.0,2.0,1.0]
o = n(x)
println(o)

-0.41 (gr: 0.0, op: +)


In [3]:
# now let's look at the tree
nodes,depth = buildgraph(o)
printgraph(nodes,depth)


Tree:
----- 

-0.41 (gr: 0.0, op: +)   
|------------------------|
-0.11 (gr: 0.0, op:  )   -0.3 (gr: 0.0, op: +)   
                         |----------------------------------------------|
                         0.47 (gr: 0.0, op: *)                          -0.77 (gr: 0.0, op: +)   
                         |----------------------|                       |-----------------------------------------------|
                         1.0 (gr: 0.0, op:  )   0.47 (gr: 0.0, op:  )   -0.31 (gr: 0.0, op: *)                          -0.45 (gr: 0.0, op: *)   
                                                                        |----------------------|                        |----------------------|
                                                                        2.0 (gr: 0.0, op:  )   -0.16 (gr: 0.0, op:  )   1.0 (gr: 0.0, op:  )   -0.45 (gr: 0.0, op:  )   

In [5]:
# you can't give the wrong number of inputs
try
    n([1.0,2.0])
catch e
    println(e)
    println("\n ^ correctly throws error")
end

ErrorException("In calling n(x), expected 3 inputs to neuron, got 2:\n\tx = [1.0, 2.0]")

 ^ correctly throws error


In [4]:
## make a relu activation neuron with three inputs and call it
# note the op here is now relu instead of +
# run this multiple times to see relu effect on a negative value
n = neuron(3,relu)
x = [1.0,2.0,1.0]
o = n(x) 

nodes,depth = buildgraph(o)
printgraph(nodes,depth)



Tree:
----- 

0.0 (gr: 0.0, op: relu)   
|
-1.6 (gr: 0.0, op: +)   
|-----------------------|
0.87 (gr: 0.0, op:  )   -2.5 (gr: 0.0, op: +)   
                        |-----------------------------------------------|
                        -0.72 (gr: 0.0, op: *)                          -1.8 (gr: 0.0, op: +)   
                        |----------------------|                        |-----------------------------------------------|
                        1.0 (gr: 0.0, op:  )   -0.72 (gr: 0.0, op:  )   -1.5 (gr: 0.0, op: *)                           -0.25 (gr: 0.0, op: *)   
                                                                        |----------------------|                        |----------------------|
                                                                        2.0 (gr: 0.0, op:  )   -0.77 (gr: 0.0, op:  )   1.0 (gr: 0.0, op:  )   -0.25 (gr: 0.0, op:  )   

### Layers
Layers are just groups of neurons, all with the same inputs (or at least here with fully connected layers). They essentially map from N dimensional input to M dimensional outputs where N is the number of inputs to each neuron in the layer and M is the number of neurons in the layer (recall each neuron has one output).

The code is here:
```julia
mutable struct Layer{T<:AbstractFloat,F<:Union{Function,Nothing}}
    neurons::Vector{Neuron{T,F}}
    inputs::Int
    outputs::Int
end
```

and again we make the layer callable:

```julia
function (l::Layer)(x)
    out = [n(x) for n in l.neurons]
    length(out) == 1 && return out[1]
    return out
end
```

In [7]:
## make a layer of a single neuron with relu activation
# this is just a layer wrapper for one neuron to make sure the API works and not break the printing
l = layer(3,1,relu)
o = l(x)

nodes,depth = buildgraph(o)
printgraph(nodes,depth)


Tree:
----- 

0.0 (gr: 0.0, op: relu)   
|
-1.0 (gr: 0.0, op: +)   
|-----------------------|
0.88 (gr: 0.0, op:  )   -1.9 (gr: 0.0, op: +)   
                        |-----------------------------------------------|
                        -0.55 (gr: 0.0, op: *)                          -1.3 (gr: 0.0, op: +)   
                        |----------------------|                        |-----------------------------------------------|
                        1.0 (gr: 0.0, op:  )   -0.55 (gr: 0.0, op:  )   -0.72 (gr: 0.0, op: *)                          -0.6 (gr: 0.0, op: *)   
                                                                        |----------------------|                        |----------------------|
                                                                        2.0 (gr: 0.0, op:  )   -0.36 (gr: 0.0, op:  )   1.0 (gr: 0.0, op:  )   -0.6 (gr: 0.0, op:  )   

### Multilayer Perceptrons

These are simply lists of layers where the output of one is the input to the next. This means the n_in and n_out have to line up in the series. 

Here's the simple definition of the MLP type:
```julia
mutable struct MLP
    layers::Vector{Layer}
end
```

The constructor takes how many inputs (from the features in the data set) and a list of how many neurons per layer and makes sure the ins and outs line up.

```julia
function mlp(n_in,n_outs,act=relu)
    n_vec = [n_in;n_outs]
    MLP([i!= length(n_outs) ? layer(n_vec[i],n_vec[i+1],act) : layer(n_vec[i],n_vec[i+1],nothing) for i in 1:length(n_outs)])
end
```



At the end, we need to make sure the output maps to the correct domain for our problem. That's why the last one has a linear output. We don't want to truncate the domain with an activation function by default.


### Fitting a NN

Instead of showing a simple MLP which is impossible to print with the tools in this package, we'll define the same one used in Micrograd to classify the sklearn moons data.

This is an MLP that takes in 2D points and predicts a binary class. It uses 3 layers of 16 (relu), 16 (relu), and 1 (linear). The ending linear layer makes sure we can actually calculate hinge loss. Try setting it to relu and notice it won't fit.

In [8]:
X,y = getmoons()
m = mlp(2, [16, 16, 1])


fit(m,X,y)

Step: 0	Loss: 1.8	Accuracy: 0.5


Step: 1	Loss: 6.9	Accuracy: 0.5


Step: 2	Loss: 0.64	Accuracy: 0.81


Step: 3	Loss: 0.73	Accuracy: 0.78


Step: 4	Loss: 0.87	Accuracy: 0.5


...

Step: 95	Loss: 0.014	Accuracy: 1.0


Step: 96	Loss: 0.014	Accuracy: 1.0


Step: 97	Loss: 0.014	Accuracy: 1.0


Step: 98	Loss: 0.014	Accuracy: 1.0


Step: 99	Loss: 0.014	Accuracy: 1.0


In [9]:
# ok there's no plotting library. this is not quite as satisfying.
y_fit = m.(X)
cat(y,sign.(getfield.(y_fit,:data)),dims=2)

100×2 Matrix{Float64}:
 -1.0  -1.0
 -1.0  -1.0
  1.0   1.0
 -1.0  -1.0
  1.0   1.0
  1.0   1.0
  1.0   1.0
  1.0   1.0
  1.0   1.0
 -1.0  -1.0
  ⋮    
  1.0   1.0
  1.0   1.0
  1.0   1.0
  1.0   1.0
  1.0   1.0
 -1.0  -1.0
  1.0   1.0
  1.0   1.0
 -1.0  -1.0